In [ ]:
library(readr)
library(sqldf)
library(ggplot2)

acidentes <- read_csv("C:/Users/User/Desktop/Mestrado Yukio 2016/Mineracao de Dados/Trabalho Final/dataset/Kaagle_Upload.csv")

par(mfrow=c(1,2))
par(las = 1)
hist(acidentes$sex_of_driver, main = "Sexo das vítimas", xlab="Sexo", ylab="", cex.lab=0.90, xlim = c(0,3))
legend("topright", c("1 - Masculino","2 - Feminino","3 - Não informado"), cex=0.80)
hist(acidentes$age_of_driver, main = "Idade dos motoristas", xlab="Idade", ylab="", cex.lab=0.90)

par(mfrow=c(1,1))
par(las = 1 )
boxplot(acidentes$age_of_driver, main="Idade do Motorista", xlab="Idade", ylab="Frequência")
abline(h = min(acidentes$age_of_driver), col="brown")
abline(h = max(acidentes$age_of_driver), col="green")
abline(h = mean(acidentes$age_of_driver), col="red")
abline(h = median(acidentes$age_of_driver), col="blue")
abline(h = quantile(acidentes$age_of_driver, c(0.25,0.75)), col="orange")

boxplot(acidentes$sex_of_driver, main="Sexo do Motorista", xlab="Sexo", ylab="Frequência")
abline(h = min(acidentes$sex_of_driver), col="brown")
abline(h = max(acidentes$sex_of_driver), col="green")
abline(h = mean(acidentes$sex_of_driver), col="red")
abline(h = median(acidentes$sex_of_driver), col="blue")
abline(h = quantile(acidentes$sex_of_driver, c(0.25,0.75)), col="orange")

counts1 <- table(acidentes$sex_of_driver, acidentes$age_of_driver)
barplot(counts1, main="Distribuição dos acidentes por Sexo vs Idade", xlab="Idade dos motoristas", col=c("red","darkblue"), legend=c("Feminino","Masculino"), beside=TRUE)

acidentesNew <- sqldf("select vehicle_type,sex_of_driver,age_of_driver,age_of_vehicle,accident_severity from acidentes where vehicle_type in (2,3,4,5,8,9,22,23) and sex_of_driver in (1,2) and age_of_driver>17 and age_of_vehicle>0 and accident_severity>0")

wss <- (nrow(acidentesNew)-1)*sum(apply(acidentesNew,2,var))
for (i in 2:15){
  wss[i] <- sum(kmeans(acidentesNew,centers = i)$withinss)
}

plot(1:15, wss, type="b", xlab="Número de clusters", ylab="Soma dos quadrados", main="Número de clusters ótimos - Método Elbow", pch=20, cex=2)

set.seed(20)

k.resultado <- kmeans(acidentesNew, centers = 4)
k.resultado$cluster

acidentesNew$cluster <- k.resultado$cluster

data <- sample(2, nrow(acidentesNew), replace = TRUE, prob=c(0.70,0.30))
dfTreino <- acidentesNew[data==1,]
dfTeste <- acidentesNew[data==2,]

k.resultado

#Plot do Treino
plot(dfTreino, col=(k.resultado$cluster +1), main="Resultado K-Means com 4 clusters - Treino", pch=20, cex=2)

dfTreinoC1 <- sqldf("select vehicle_type,sex_of_driver,age_of_driver,age_of_vehicle,accident_severity from dfTreino where cluster=1")
dfTreinoC2 <- sqldf("select vehicle_type,sex_of_driver,age_of_driver,age_of_vehicle,accident_severity from dfTreino where cluster=2")
dfTreinoC3 <- sqldf("select vehicle_type,sex_of_driver,age_of_driver,age_of_vehicle,accident_severity from dfTreino where cluster=3")
dfTreinoC4 <- sqldf("select vehicle_type,sex_of_driver,age_of_driver,age_of_vehicle,accident_severity from dfTreino where cluster=4")
dfTreinoC5 <- sqldf("select vehicle_type,sex_of_driver,age_of_driver,age_of_vehicle,accident_severity from dfTreino where cluster=5")

summary(dfTreinoC1)
summary(dfTreinoC2)
summary(dfTreinoC3)
summary(dfTreinoC4)
summary(dfTreinoC5)

#Plot do Teste
plot(dfTeste, col=(k.resultado$cluster +1), main="Resultado K-Means com 4 clusters - Teste", pch=20, cex=2)

dfTesteC1 <- sqldf("select vehicle_type,sex_of_driver,age_of_driver,age_of_vehicle,accident_severity from dfTeste where cluster=1")
dfTesteC2 <- sqldf("select vehicle_type,sex_of_driver,age_of_driver,age_of_vehicle,accident_severity from dfTeste where cluster=2")
dfTesteC3 <- sqldf("select vehicle_type,sex_of_driver,age_of_driver,age_of_vehicle,accident_severity from dfTeste where cluster=3")
dfTesteC4 <- sqldf("select vehicle_type,sex_of_driver,age_of_driver,age_of_vehicle,accident_severity from dfTeste where cluster=4")
dfTesteC5 <- sqldf("select vehicle_type,sex_of_driver,age_of_driver,age_of_vehicle,accident_severity from dfTeste where cluster=5")

summary(dfTesteC1)
summary(dfTesteC2)
summary(dfTesteC3)
summary(dfTesteC4)
summary(dfTesteC5)